In [1]:
import os
from dotenv import load_dotenv
from langchain_mistralai import ChatMistralAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
load_dotenv()

api_key = os.environ.get("MISTRAL_API_KEY")
if not api_key:
    raise ValueError("MISTRAL_API_KEY is not set or could not be loaded!")

model = ChatMistralAI(model="mistral-large-latest", api_key=api_key)

import requests
from pydantic import BaseModel, Field
import datetime

from langchain.agents import tool

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec
from langchain.tools.render import format_tool_to_openai_tool

#creating list of both functions specs.
tools = [search_wikipedia, get_current_temperature]

from langchain_core.messages import HumanMessage
model_with_tools = model.bind_tools(tools)

from langchain import hub

# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")

from langchain.agents import create_tool_calling_agent

agent = create_tool_calling_agent(model, tools, prompt)

from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

agent_executor.invoke({"input": "Hi, My Name is Krunal. I live in Surat"})



f:\LangChain Test\venv\lib\site-packages\langsmith\client.py:241: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(




> Entering new AgentExecutor chain...
Hello Krunal, nice to meet you! How can I assist you today?

> Finished chain.


{'input': 'Hi, My Name is Krunal. I live in Surat',
 'output': 'Hello Krunal, nice to meet you! How can I assist you today?'}

In [2]:
agent_executor.invoke({"input": "Do you know my name?"})



> Entering new AgentExecutor chain...
No, I do not know your name. Would you like to tell me?

> Finished chain.


{'input': 'Do you know my name?',
 'output': 'No, I do not know your name. Would you like to tell me?'}

# Adding Memory to the Agent

In [5]:
agent_executor.invoke({"input": "Hi, My Name is Krunal. I live in Surat", "chat_history": []})
from langchain_core.messages import AIMessage, HumanMessage
agent_executor.invoke(
    {
        "chat_history": [
            HumanMessage(content="Hi, My Name is Krunal. I live in Surat"),
            AIMessage(content="Hello Krunal, nice to meet you! How can I assist you today?"),
        ],
        "input": "what's my name and where do i live?",
    }
)



> Entering new AgentExecutor chain...
Hello Krunal, nice to meet you! How can I assist you today?

> Finished chain.


> Entering new AgentExecutor chain...
Your name is Krunal and you live in Surat.

> Finished chain.


{'chat_history': [HumanMessage(content='Hi, My Name is Krunal. I live in Surat', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello Krunal, nice to meet you! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'input': "what's my name and where do i live?",
 'output': 'Your name is Krunal and you live in Surat.'}

> To keep track of these messages automatically, we can wrap this in a "RunnableWithMessageHistory"!

In [9]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

In [10]:
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="input", 
    history_messages_key="chat_history", 
)

In [11]:
agent_with_chat_history.invoke(
    {"input": "Hi, My Name is Krunal. I live in Surat"},
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Hello Krunal, I'm here to help you. How are you today?

> Finished chain.


{'input': 'Hi, My Name is Krunal. I live in Surat',
 'chat_history': [],
 'output': "Hello Krunal, I'm here to help you. How are you today?"}

In [12]:
agent_with_chat_history.invoke(
    {"input": "what's my name and where do i live?"},
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...
Your name is Krunal and you live in Surat.

> Finished chain.


{'input': "what's my name and where do i live?",
 'chat_history': [HumanMessage(content='Hi, My Name is Krunal. I live in Surat', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hello Krunal, I'm here to help you. How are you today?", additional_kwargs={}, response_metadata={})],
 'output': 'Your name is Krunal and you live in Surat.'}

In [13]:
agent_with_chat_history.invoke(
    {"input": "what's the current weather in my city?"},
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...

Invoking: `get_current_temperature` with `{'latitude': 21.1702, 'longitude': 72.8311}`


The current temperature is 26.3°CGreat! If you need more information or assistance with anything else, just let me know. I'm here to help.

> Finished chain.


{'input': "what's the current weather in my city?",
 'chat_history': [HumanMessage(content='Hi, My Name is Krunal. I live in Surat', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hello Krunal, I'm here to help you. How are you today?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="what's my name and where do i live?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Krunal and you live in Surat.', additional_kwargs={}, response_metadata={})],
 'output': "Great! If you need more information or assistance with anything else, just let me know. I'm here to help."}

In [14]:
agent_with_chat_history.invoke(
    {"input": "Can you get me brief history about my city?"},
    config={"configurable": {"session_id": "<foo>"}},
)



> Entering new AgentExecutor chain...

Invoking: `search_wikipedia` with `{'query': 'surat gujrat india history'}`


Page: Gujarat
Summary: Gujarat ( GUUJ-ə-RAHT; ISO: Gujarāt, Gujarati: [ˈɡudʒəɾat̪] ) is a state along the western coast of India. Its coastline of about 1,600 km (990 mi) is the longest in the country, most of which lies on the Kathiawar peninsula. Gujarat is the fifth-largest Indian state by area, covering some 196,024 km2 (75,685 sq mi); and the ninth-most populous state, with a population of 60.4 million in 2011. It is bordered by Rajasthan to the northeast, Dadra and Nagar Haveli and Daman and Diu to the south, Maharashtra to the southeast, Madhya Pradesh to the east, and the Arabian Sea and the Pakistani province of Sindh to the west. Gujarat's capital city is Gandhinagar, while its largest city is Ahmedabad. The Gujaratis are indigenous to the state and their language, Gujarati, is the state's official language.
The state encompasses 23 sites of the ancient Indus

{'input': 'Can you get me brief history about my city?',
 'chat_history': [HumanMessage(content='Hi, My Name is Krunal. I live in Surat', additional_kwargs={}, response_metadata={}),
  AIMessage(content="Hello Krunal, I'm here to help you. How are you today?", additional_kwargs={}, response_metadata={}),
  HumanMessage(content="what's my name and where do i live?", additional_kwargs={}, response_metadata={}),
  AIMessage(content='Your name is Krunal and you live in Surat.', additional_kwargs={}, response_metadata={}),
  HumanMessage(content="what's the current weather in my city?", additional_kwargs={}, response_metadata={}),
  AIMessage(content="Great! If you need more information or assistance with anything else, just let me know. I'm here to help.", additional_kwargs={}, response_metadata={})],
 'output': "Surat is a city in the Indian state of Gujarat. It is known for its historical significance, particularly during the Mughal Empire. The city was a prominent port and trading cente